<a href="https://colab.research.google.com/github/ddebashish21/Website-AI-Assistant/blob/main/RAG_Application_to_interact_with_websites_using_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RAG Application to interact with websites using Large Language Model**

The aim of this project is to create a RAG application to interact with the websites by feeding the url of the websites to the model, which uses an LLM to respond to the queries asked by the user.

**Setting up the environment using langchain**

In [59]:
%%capture
!pip install langchain_community

**Installing and importing the necessary libraries to extract the data of the websites**

In [60]:
!pip -q install unstructured

In [61]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls=["https://www.odinschool.com/", "https://www.odinschool.com/data-science-course"] # The websites we want to interact to
loader=UnstructuredURLLoader(urls=urls)
data=loader.load() # Extracting the data from the websites and storing it in data variable

**Splitting the data of the websites into chunks**

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_chunks=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks=text_chunks.split_documents(data) # Dividing the whole data into chunks
print(len(chunks)) # Printing the length of the chunks created

51


**Installing Langchain HuggingFace to use HuggingFace embedding and llm models**

In [63]:
!pip -q install langchain_huggingface

In [64]:
# Importing the embedding model from HuggingFace to embed the data
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

**Installing a Vector DataBase to store the embedded data**

In [65]:
!pip -q install faiss-cpu

In [66]:
from langchain.vectorstores import FAISS
db=FAISS.from_documents(chunks,embeddings) # Storing the embedded data in db variable

**Setting up the LLM model to generate text and answer user queries based on the information extracted from the websites**

In [67]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM, pipeline
tokenizer=AutoTokenizer.from_pretrained("google/flan-t5-base") # Tokenizing the data using pre-trained tokenizer for the llm model
model=AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base") # Loading the pre-trained llm model
pipe=pipeline("text2text-generation",model=model,tokenizer=tokenizer, max_new_tokens=500) # Setting up the model

In [68]:
from langchain_huggingface import HuggingFacePipeline
llm=HuggingFacePipeline(pipeline=pipe)

In [69]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=db.as_retriever()) # Retrieving the information from the vector database using llm

In [70]:
query="what are the courses that odinschool provide?" # The query of the user

In [71]:
response=qa.invoke(query) # Loading the response of the model for the given query

Token indices sequence length is longer than the specified maximum sequence length for this model (857 > 512). Running this sequence through the model will result in indexing errors


In [72]:
print(response['result']) # Printing the response

Data Science Courses


# **Creating a User Interface using Gradio to ask questions to the model**

In [73]:
!pip -q install gradio

In [75]:
import gradio as gr

def virtualAssistant(question, history):
  response=qa.invoke(question)
  return response['result']

gr.ChatInterface(
    virtualAssistant,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Virtual Assistant of the website",
    description="Ask virtual sssistant any question",
    theme="soft",
    examples=["What is the duration of the Data Science course?", "How many companies come for the recruitment?", "What is the tuition fee for Data Science Course?","Contact details"],
    cache_examples=True
).launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Caching examples at: '/content/.gradio/cached_examples/206'
Caching example 1/4
Caching example 2/4
Caching example 3/4
Caching example 4/4
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e361689913cdc386e8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
